In [ ]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.datasets
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,mean_absolute_error,mean_squared_log_error
from sklearn.linear_model import LogisticRegression

from sklearn import tree
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from IPython.display import Image



import numpy as np
import pandas as pd
import scipy
from scipy import stats

In [ ]:
df = pd.read_csv('Clothing_Store.csv')
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('&', 'and')
df.columns = df.columns.str.replace('-', '_')
df.drop([6268], inplace = True)

In [ ]:
df["CreditCard_User"].describe()
df["WEB"].sum()
x=0
for i in df["MAILED"]:
    if i > 0:
        x=x+1
print(x)

In [ ]:
purchases=df["Num_Purchases"]
total_spend=df["TotalSpend"]
print(type(purchases))
print(df[0:0])

In [ ]:
#שינוי עמודת הכרטיס אשראי לבינארית
df["CreditCard_User"]=np.where(df["CreditCard_User"]=="Yes",1,0)

In [ ]:
#הטבלה עם העמודות שמעניינות אותנו באמת
#df=df.drop(columns="HHKEY","ZIP_CODE","Credit_Card_User","Average_Spend","PC_CALC20"]
df_reg=df[["TotalSpend","PROMOS","COUPONS","WEB","MAILED","RESPONDED","Perc_Returns","CreditCard_User","DiversitySpend"]]
pd.set_option('display.max_columns', 35) #replace n with the number of columns you want to see completely
df_corr=df_reg.corr()
print(df_corr)
df_reg=df_reg.drop(columns="RESPONDED")

In [ ]:
#הרגרסיה
X=df_reg.drop(columns="TotalSpend")
y=df_reg["TotalSpend"]
training_set_fraction = 0.7
msk = np.random.rand(len(y)) < training_set_fraction
X_train = X[msk]
y_train = y[msk]
X_test = X[~msk]
y_test = y[~msk]
print ('Training set size: {0:d}\nTest set size: {1:d}'.format(len(X_train),len(X_test)))
lm = LinearRegression()
lm.fit(X_train, y_train)
print('Regression finished with R^2=',lm.score(X_test,y_test))
y_hat=lm.predict(X_test)
mae=mean_absolute_error(y_hat,y_test)
mse=mean_squared_error(y_hat,y_test)
rmae=mae**(0.5)
rmse=mse**(0.5)
print("root of mse=",rmse)
importance=lm.coef_
the_brand_is_right = pd.Series(importance, index=X.columns)
sorted_importances=the_brand_is_right.sort_values()
print(sorted_importances)

In [ ]:
#יצירת שתי טבלאות חדשות למתחת לחציון ומעל החציון
median_total_spend=df["TotalSpend"].median()
print(median_total_spend)
df["bigger_than_median"]=np.where(df["TotalSpend"]>=median_total_spend,1,0)
df_less_median=df.loc[df.bigger_than_median==0]
df_bigger_median=df.loc[df.bigger_than_median==1]

In [ ]:
#t test
stats.ttest_ind(df_less_median["COUPONS"],df_bigger_median["COUPONS"],equal_var=False)
high_pvalue=[]
low_pvalue=[]
mean_bigger=[]
mean_less=[]
subject=[]
p_value=[]
for r in ["PROMOS","WEB","COUPONS","MAILED","RESPONDED","DiversitySpend","Perc_Returns"]:
    result=stats.ttest_ind(df_less_median[r],df_bigger_median[r],equal_var=False)
    if (result.pvalue < 0.05):
        high_pvalue.append(r)
        p_value.append("yes")
    else:
        low_pvalue.append(r)
        p_value.append("no")
    mean_bigger.append(df_bigger_median[r].mean())
    mean_less.append(df_less_median[r].mean())
    subject.append(r)
for r in ["PSWEATERS", "PKNIT_TOPS", "PKNIT_DRES", "PBLOUSES", "PJACKETS", "PCAR_PNTS", "PCAS_PNTS", "PSHIRTS", "PDRESSES", "PSUITS", "POUTERWEAR", "PJEWELRY", "PFASHION", "PLEGWEAR"]:
    result=stats.ttest_ind(df_less_median[r],df_bigger_median[r],equal_var=False)
    if (result.pvalue < 0.05):
        high_pvalue.append(r)
        p_value.append("yes")
    else:
        low_pvalue.append(r)
        p_value.append("no")
    mean_bigger.append(df_bigger_median[r].mean())
    mean_less.append(df_less_median[r].mean())
    subject.append(r)
print("high p value are: ",high_pvalue)
print("low p value are: ",low_pvalue)
d_means={"subject": subject, "mean_bigger": mean_bigger, "mean_less": mean_less, "significance": p_value}
df_means=pd.DataFrame(data=d_means)
df_means

    

In [ ]:
#רגרסיה ליניארית על מתחת לחציון
df_reg=df_less_median[["TotalSpend","CreditCard_User", "PROMOS", "WEB", "COUPONS", "MAILED","Perc_Returns"]]


In [ ]:
#רגרסיה ליניארית על הכל
df_reg=df[["TotalSpend","CreditCard_User", "PROMOS", "WEB", "COUPONS", "MAILED","Perc_Returns"]]
X=df_reg.drop(columns="TotalSpend")
y=df_reg["TotalSpend"]
training_set_fraction = 0.7
msk = np.random.rand(len(y)) < training_set_fraction
X_train = X[msk]
y_train = y[msk]
X_test = X[~msk]
y_test = y[~msk]
print ('Training set size: {0:d}\nTest set size: {1:d}'.format(len(X_train),len(X_test)))
lm = LinearRegression()
lm.fit(X_train, y_train)
print('Regression finished with R^2=',lm.score(X_test,y_test))
y_hat=lm.predict(X_test)
mae=mean_absolute_error(y_hat,y_test)
mse=mean_squared_error(y_hat,y_test)
rmae=mae**(0.5)
rmse=mse**(0.5)
print("root of mse=",rmse)

In [ ]:
pd.set_option('display.max_columns', 35) #replace n with the number of columns you want to see completely
df_describe=df.describe()
print(df_describe)


In [ ]:
#sort of the means and the medians from the describe
means=df_describe.iloc[1,6:20]
means.sort_values(ascending=False)
medians=df_describe.iloc[5,6:20]
medians.sort_values(ascending=False)

In [ ]:
total_spend.plot.hist(by=None, bins=1000)
bigger_than_5000=0
bigger_than_1000=0
lower_than_100=0
for r in total_spend:
    if r > 5000:
        bigger_than_5000+=1
    if r > 1000:
        bigger_than_1000+=1
    if r < 100:
        lower_than_100+=1
        
bigger_than_5000=bigger_than_5000*100/len(total_spend)
bigger_than_1000=bigger_than_1000*100/len(total_spend)
lower_than_100=lower_than_100*100/len(total_spend)

print("percent of clients that buy in more than 5000 dollars: ",bigger_than_5000)
print("percent of clients that buy in more than 1000 dollars: ",bigger_than_1000)
print("percent of clients that buy in less than 100 dollars: ",lower_than_100)
print 

In [ ]:
print(df.head())
print(df["ZIP_CODE"])